## Final Project

!pip install boto3 python-dotenv

In [1]:
import os
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Set up AWS credentials
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")

In [3]:
# Create a session using the credentials
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

# Create an S3 client
s3 = session.client('s3', config=Config(signature_version='s3v4'))

# Specify your bucket name and file name
bucket_name = 'us-accidents-final'
file_name = 'US_Accidents_March23.csv'

# Verify the bucket contents
try:
    response = s3.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in response:
        print("Files in the bucket:")
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("Bucket is empty or does not exist.")
except ClientError as e:
    print(f"Error listing objects in bucket: {e}")



Files in the bucket:
US_Accidents_March23.csv


In [4]:
# Download the file to notebook
s3_bucket = s3.download_file(bucket_name, file_name, file_name)
print(s3_bucket)




None


In [5]:
# Load the file into a Pandas DataFrame
import pandas as pd
data_df = pd.read_csv(file_name)
data_df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


## DATA CLEANING

In [6]:
#Running info() functiont to understand the makeup of the dataframe
data_df.info()
print('-------')
print(len(data_df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [7]:
#validating if there are any duplicated values
print(f'there are: {data_df.duplicated().sum()} duplicated values in the dataset')

there are: 0 duplicated values in the dataset


In [8]:
# Inspecting the dataframe for null values
na_counts = data_df.isna().sum()
print(na_counts)

ID                             0
Source                         0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  3402762
End_Lng                  3402762
Distance(mi)                   0
Description                    5
Street                     10869
City                         253
County                         0
State                          0
Zipcode                     1915
Country                        0
Timezone                    7808
Airport_Code               22635
Weather_Timestamp         120228
Temperature(F)            163853
Wind_Chill(F)            1999019
Humidity(%)               174144
Pressure(in)              140679
Visibility(mi)            177098
Wind_Direction            175206
Wind_Speed(mph)           571233
Precipitation(in)        2203586
Weather_Condition         173459
Amenity                        0
Bump      

**Discussion on dropping records/columns** Based on the above results, there are some columns with significant numbers of null/na values. Given that we do not believe the are relationships (linear or otherwise) between the data items, and given that each record represents a different event, which may have occured in a different state at a different time, under different weather conditions, we will choose to either drop the columns with significant missing values (subject to our assessment of their significance to our objective) or drop the rows/records with missing values (whose columns we have deeped important to our objective and thus will keep). While this will greatly reduce the number of records, we are confident that the remaining data will be of a higher quality and will result in better modeling. 

In [9]:
#filtering for columns where result is greater than 1 
filterd_na_counts = na_counts[na_counts>1]

#displaying the result
print(filterd_na_counts)

End_Lat                  3402762
End_Lng                  3402762
Description                    5
Street                     10869
City                         253
Zipcode                     1915
Timezone                    7808
Airport_Code               22635
Weather_Timestamp         120228
Temperature(F)            163853
Wind_Chill(F)            1999019
Humidity(%)               174144
Pressure(in)              140679
Visibility(mi)            177098
Wind_Direction            175206
Wind_Speed(mph)           571233
Precipitation(in)        2203586
Weather_Condition         173459
Sunrise_Sunset             23246
Civil_Twilight             23246
Nautical_Twilight          23246
Astronomical_Twilight      23246
dtype: int64


Based on the above results. We Will trim the data to remove: 
**1. columns that we do not deem useful based on our objectives and related ratioanale** 
- End_lat: many missing values. We we have full beginning lat values. 
- End_Lng: many missing values. We we have full beginning Lng values.
- Airport_Code: not significant to our analysis - mainly used for weather reasons. 
- Weather_Timestamp: not significant to our analysis. We have beginning time and end time. 
- Wind_Chill(F): not significant to our analysis, we have weather_condition. 
- Precipitation(in): we have weather_condition
- Nautical_Twilight: Will use Civil_Twilight
- Astronomical_Twilight: Will use Civil_Twilight

and, 
**2. Rows with null values that will potentially skew the results. In this case we only drop the rows related to the missing values**: 
- Description
- Street
- City
- Zipcode
- Timezone
- Temperature
- Humidity(%)
- Pressure(in)
- Visibility(mi)
- Wind_Direction
- Wind_Speed(mph)
- Weather_Condition
- Sunrise_Sunset             
- Civil_Twilight

In [10]:
len(data_df)

7728394

In [11]:
# Dropping Columns first

clean_data = data_df.drop(columns = ['End_Lat','End_Lng','Airport_Code','Weather_Timestamp','Wind_Chill(F)',
                     'Precipitation(in)','Nautical_Twilight','Astronomical_Twilight'], inplace=False)

clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 38 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 object 
 1   Source             object 
 2   Severity           int64  
 3   Start_Time         object 
 4   End_Time           object 
 5   Start_Lat          float64
 6   Start_Lng          float64
 7   Distance(mi)       float64
 8   Description        object 
 9   Street             object 
 10  City               object 
 11  County             object 
 12  State              object 
 13  Zipcode            object 
 14  Country            object 
 15  Timezone           object 
 16  Temperature(F)     float64
 17  Humidity(%)        float64
 18  Pressure(in)       float64
 19  Visibility(mi)     float64
 20  Wind_Direction     object 
 21  Wind_Speed(mph)    float64
 22  Weather_Condition  object 
 23  Amenity            bool   
 24  Bump               bool   
 25  Crossing          

In [12]:
#Dropping Na/Null records related to specified columns

clean_data.dropna(subset = ['Description','Street','City','Zipcode','Temperature(F)','Humidity(%)','Pressure(in)',
                      'Visibility(mi)','Wind_Direction','Wind_Speed(mph)','Weather_Condition','Sunrise_Sunset','Civil_Twilight'], inplace=True)

In [13]:
# Rechecking if there are any null values left in the dataset
clean_data.isna().sum()

ID                   0
Source               0
Severity             0
Start_Time           0
End_Time             0
Start_Lat            0
Start_Lng            0
Distance(mi)         0
Description          0
Street               0
City                 0
County               0
State                0
Zipcode              0
Country              0
Timezone             0
Temperature(F)       0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Direction       0
Wind_Speed(mph)      0
Weather_Condition    0
Amenity              0
Bump                 0
Crossing             0
Give_Way             0
Junction             0
No_Exit              0
Railway              0
Roundabout           0
Station              0
Stop                 0
Traffic_Calming      0
Traffic_Signal       0
Turning_Loop         0
Sunrise_Sunset       0
Civil_Twilight       0
dtype: int64

In [14]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7051556 entries, 2 to 7728393
Data columns (total 38 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 object 
 1   Source             object 
 2   Severity           int64  
 3   Start_Time         object 
 4   End_Time           object 
 5   Start_Lat          float64
 6   Start_Lng          float64
 7   Distance(mi)       float64
 8   Description        object 
 9   Street             object 
 10  City               object 
 11  County             object 
 12  State              object 
 13  Zipcode            object 
 14  Country            object 
 15  Timezone           object 
 16  Temperature(F)     float64
 17  Humidity(%)        float64
 18  Pressure(in)       float64
 19  Visibility(mi)     float64
 20  Wind_Direction     object 
 21  Wind_Speed(mph)    float64
 22  Weather_Condition  object 
 23  Amenity            bool   
 24  Bump               bool   
 25  Crossing           bool

In [15]:
print(f'we reduced the total records by a total of {len(data_df) - len(clean_data)} rows, and total columns by 8')

we reduced the total records by a total of 676838 rows, and total columns by 8


## DATA ANALYSIS AND VISUALIZATION

## MODELING 

In [16]:
 # Import the required modules for supervised learning
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# Unspervised learning
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt 

import hvplot.pandas
from sklearn.cluster import KMeans

In [17]:
 # Seperate the features, X,  from the target variable, y
y = clean_data['Severity']
X = clean_data.drop(columns='Severity')

In [18]:
X.head()

,ID,Source,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,...,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight
2,A-3,Source2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,...,False,False,False,False,False,False,True,False,Night,Night
3,A-4,Source2,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,...,False,False,False,False,False,False,False,False,Night,Day
4,A-5,Source2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,...,False,False,False,False,False,False,True,False,Day,Day
5,A-6,Source2,2016-02-08 07:44:26,2016-02-08 08:14:26,40.100590,-82.925194,0.01,Accident on I-270 Outerbelt Northbound near Ex...,Westerville Rd,Westerville,...,False,False,False,False,False,False,False,False,Day,Day
6,A-7,Source2,2016-02-08 07:59:35,2016-02-08 08:29:35,39.758274,-84.230507,0.00,Accident on Oakridge Dr at Woodward Ave. Expec...,N Woodward Ave,Dayton,...,False,False,False,False,False,False,False,False,Day,Day


In [20]:
# Preview the first five entries for the target variable
y[:10]

2     2
3     3
4     2
5     3
6     2
7     3
8     2
9     3
10    3
11    3
Name: Severity, dtype: int64